In [8]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import uuid
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine, update, Table, Column, Integer, String, MetaData, inspect,text
import numpy as np
import json
import pyodbc

SGN_EXTENSION = ".sgn"
XML_EXTENSION = ".xml"
pathFolder = r"C:\Users\rinlapas\Desktop\202309_September_xml"

# Database configuration
db_config = {
    'dbname': 'import_final',
    'user': 'postgres',
    'password': '123',
    'host': 'localhost',
    'port': '5432'
}

postgres_db_config_med = {
    'dbname': 'meb',
    'user': 'postgres',
    'password': 'MyTTT%401234',
    'host': '203.154.82.165',
    'port': '5432'
}

sql_server_config_server = {
    'driver': '{SQL Server}',
    'server': '203.154.82.165',
    'database': 'response',
    'user': 'sa',
    'password': 'MyTTT@1234'
}

def setup_postgres_engine(db_config):
    try:
        db_url = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}"
        engine = create_engine(db_url)
        print("PostgreSQL connection successful")
        return engine
    except Exception as e:
        print(f"Error connecting to PostgreSQL: {e}")
        return None

def setup_sql_server_connection(db_config):
    try:
        conn_str = (
            f"DRIVER={db_config['driver']};"
            f"SERVER={db_config['server']};"
            f"DATABASE={db_config['database']};"
            f"UID={db_config['user']};"
            f"PWD={db_config['password']};"
        )
        conn = pyodbc.connect(conn_str)
        print("SQL Server connection successful")
        return conn
    except pyodbc.Error as e:
        print(f"Error connecting to SQL Server: {e}")
        return None

# Establish connections
sql_server_resp = setup_sql_server_connection(sql_server_config_server)
postgres_mybase = setup_postgres_engine(db_config)
postgres_meb = setup_postgres_engine(postgres_db_config_med)

SQL Server connection successful
PostgreSQL connection successful
PostgreSQL connection successful


In [6]:
def process_xml_file(full_path, extension):
    data = []
    try:
        with open(full_path, 'rb') as file:
            tree = ET.parse(file, parser=ET.XMLParser(encoding='utf-8'))
        xml_root = tree.getroot()

        if xml_root is None or len(list(xml_root)) == 0:
            raise ValueError("Empty or invalid XML/SGN file")

        outermost_tag = xml_root.tag.split('}')[1] if '}' in xml_root.tag else xml_root.tag
        if extension == SGN_EXTENSION:
            data.append(process_element_sgn(xml_root, outermost_tag=outermost_tag, filename=os.path.basename(full_path)))
        elif extension == XML_EXTENSION:
            data.append(process_element_xml(xml_root, outermost_tag=outermost_tag, filename=os.path.basename(full_path)))

    except Exception as e:
        print(f"Error processing file {os.path.basename(full_path)}: {str(e)}")

    return data

def process_element_sgn(element, parent_prefix="", outermost_tag="", filename=""):
    data = {}
    for child in element:
        tag = child.tag.split('}')[1] if '}' in child.tag else child.tag
        full_tag = f"{parent_prefix}_{tag}" if parent_prefix else tag
        full_tag = full_tag.replace("DocumentControl_", "").replace("GoodsShipment_", "")
        last_tag = full_tag if not full_tag.startswith("DocumentControl_") else full_tag[len("DocumentControl_"):]
        text = child.text.strip() if child.text else None
        if text:
            data[last_tag] = text
        child_data = process_element_sgn(child, parent_prefix=full_tag, outermost_tag=outermost_tag, filename=filename)
        data.update(child_data)
    data['file_name'] = filename
    return data

def process_element_xml(element, parent_prefix="", outermost_tag="", filename=""):
    data = {}
    for child in element:
        tag = child.tag.split('}')[1] if '}' in child.tag else child.tag
        full_tag = f"{parent_prefix}_{tag}" if parent_prefix else tag
        full_tag = full_tag.replace("DocumentControl_", "").replace("GoodsShipment_", "")
        last_tag = full_tag.rsplit('_', 1)[-1]
        text = child.text.strip() if child.text else None
        if text:
            data[last_tag] = text
        child_data = process_element_xml(child, parent_prefix=full_tag, outermost_tag=outermost_tag, filename=filename)
        data.update(child_data)
    data['file_name'] = filename
    return data

def insert_data_to_database(data, table_name='response_import'):
    if data:
        df_xml = pd.DataFrame(data)
        required_columns = ['MessageType', 'file_name', 'ReferenceNumber', 'RegistrationID', 'DeclarationNumber', 'Message']
        df_xml['data'] = df_xml.drop(columns=required_columns).apply(lambda x: x.dropna().to_dict(), axis=1)
        df_xml = df_xml[required_columns + ['data']]
        df_xml = df_xml.rename(columns={'MessageType': 'res_message_type',
                                        'file_name': 'res_filename',
                                        'ReferenceNumber': 'res_ref_no',
                                        'RegistrationID': 'res_registration_id',
                                        'DeclarationNumber': 'res_declaration_number',
                                        'Message': 'res_message',
                                        'data': 'res_data'})

        df_xml = df_xml.loc[df_xml['res_message_type'].isin(['IDCA', 'PMTA'])]
        datetime_str = df_xml['res_filename'].str.split("_").str[6]
        df_xml['res_datetime'] = pd.to_datetime(datetime_str, format='%Y%m%d%H%M%S')

        # Convert res_data to JSON string
        df_xml['res_data'] = df_xml['res_data'].apply(json.dumps)
        df_xml.to_sql(table_name, postgres_mybase, if_exists='append', index=False)

def process_subfolders(pathFolder):
    subfolders = next(os.walk(pathFolder))[1]

    for subfolder in subfolders:
        subfolder_path = os.path.join(pathFolder, subfolder)
        xml_status = []
        for subroot, _, subfiles in os.walk(subfolder_path):
            for filename in subfiles:
                if filename.endswith(XML_EXTENSION) or filename.endswith(SGN_EXTENSION):
                    full_path = os.path.join(subroot, filename)
                    extension = os.path.splitext(filename)[1]
                    counts = process_xml_file(full_path, extension)
                    xml_status.extend(counts) 

        insert_data_to_database(xml_status)
        print(f"Processed subfolder: {subfolder}")

# Run the process
process_subfolders(pathFolder)

Processed subfolder: 20230901
Processed subfolder: 20230902
Processed subfolder: 20230903
Processed subfolder: 20230904
Processed subfolder: 20230905
Processed subfolder: 20230906
Processed subfolder: 20230907
Processed subfolder: 20230908
Processed subfolder: 20230909
Processed subfolder: 20230910
Processed subfolder: 20230911
Processed subfolder: 20230912
Processed subfolder: 20230913
Processed subfolder: 20230914
Processed subfolder: 20230915
Processed subfolder: 20230916
Processed subfolder: 20230917
Processed subfolder: 20230918
Processed subfolder: 20230919
Processed subfolder: 20230920
Processed subfolder: 20230921
Processed subfolder: 20230922
Processed subfolder: 20230923
Processed subfolder: 20230924
Processed subfolder: 20230925
Processed subfolder: 20230926
Processed subfolder: 20230927
Processed subfolder: 20230928
Processed subfolder: 20230929
Processed subfolder: 20230930


In [9]:
# Query InboxId from SQL Server response
query = """SELECT "FileName", "InboxId" FROM response;"""
df_sql_resp = pd.read_sql(query, sql_server_resp)

# Query reftomessageid from PostgreSQL meb_inbox
query = """SELECT "id", "reftomessageid", "messagetimestamp" FROM meb_inbox;"""
df_meb_inbox = pd.read_sql(query, postgres_meb)

# Query existing response_import table
df_response_import = pd.read_sql("SELECT * FROM response_import", postgres_mybase)

# Change data type of 'res_inbox_id' to string (text)
df_meb_inbox = df_meb_inbox.astype(str)
df_sql_resp = df_sql_resp.astype(str)


C:\Users\rinlapas\AppData\Local\Temp\ipykernel_2256\2777821023.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql_resp = pd.read_sql(query, sql_server_resp)


In [10]:
# Query existing response_import table
df_response_import = pd.read_sql("SELECT * FROM response_import", postgres_mybase)


In [11]:
# Mapping InboxId
map_inbox_id = df_sql_resp.set_index('FileName')['InboxId'].to_dict()
df_response_import['res_inbox_id'] = df_response_import['res_filename'].map(map_inbox_id)

# Mapping reftomessageid
map_message_id = df_meb_inbox.set_index('id')['reftomessageid'].to_dict()
df_response_import['res_message_id'] = df_response_import['res_inbox_id'].map(map_message_id)

In [12]:
df_response_import

,res_id,res_message_type,res_ref_no,res_declaration_number,res_message,res_data,res_datetime,res_filename,res_ref_id,res_registration_id,res_inbox_id,res_message_id
0,None,PMTA,DAFW000024980,A0300660810158,ได้รับการชำระเงินเรียบร้อยแล้ว,"{""Status"": ""A"", ""PaymentType"": ""PMT"", ""Payment...",2023-09-01 03:22:10+00:00,ebxml_DAFW_RES_RESPONSE_A_DAFW000024980_202309...,None,TH0109940001630110000010021P8,NaN,NaN
1,None,IDCA,DAFW000024980,A0300660810158,ไม่ต้องตรวจสอบพิกัดศุลกากร ราคาและของไปรับของ ...,"{""Status"": ""00"", ""TotalTax"": ""14869.00"", ""Tota...",2023-09-01 03:22:11+00:00,ebxml_DAFW_RES_RESPONSE_A_DAFW000024980_202309...,None,TH0109940001630110000010021P8,668877,NaN
2,None,IDCA,DAFW000024987,A0310660801284,ส่งมอบสินค้า,"{""Status"": ""00"", ""TotalTax"": ""0.00"", ""TotalDep...",2023-09-01 06:20:48+00:00,ebxml_DAFW_RES_RESPONSE_A_DAFW000024987_202309...,None,TH0109940001630110000010021P8,675538,NaN
3,None,IDCA,DAFW000024989,A0310660805191,ส่งมอบสินค้า,"{""Status"": ""00"", ""TotalTax"": ""1993958.00"", ""To...",2023-09-01 08:21:11+00:00,ebxml_DAFW_RES_RESPONSE_A_DAFW000024989_202309...,None,TH0109940001630110000010021P8,681243,NaN
4,None,IDCA,DAFW000024994,A0010660904245,รอการตัดบัญชีธนาคาร,"{""Status"": ""00"", ""TotalTax"": ""104736.00"", ""Tot...",2023-09-01 02:35:15+00:00,ebxml_DAFW_RES_RESPONSE_A_DAFW000024994_202309...,None,TH0109940001630110000010021P8,666426,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
164808,None,IDCA,DAFW000024972,A0290660811272,ส่งมอบสินค้า,"{""Status"": ""00"", ""TotalTax"": ""54007.00"", ""Tota...",2023-09-01 02:37:17+00:00,ebxml_DAFW_RES_RESPONSE_A_DAFW000024972_202309...,None,TH0109940001630110000010021P8,666506,NaN
164809,None,IDCA,DAFW000024972,A0290660811272,ส่งมอบสินค้า,"{""Status"": ""00"", ""TotalTax"": ""54007.00"", ""Tota...",2023-09-01 02:37:17+00:00,ebxml_DAFW_RES_RESPONSE_A_DAFW000024972_202309...,None,TH0109940001630110000010021P8,NaN,NaN
164810,None,IDCA,DAFW000024977,A0290660810673,ส่งมอบสินค้า,"{""Status"": ""00"", ""TotalTax"": ""33764.00"", ""Tota...",2023-09-01 04:17:47+00:00,ebxml_DAFW_RES_RESPONSE_A_DAFW000024977_202309...,None,TH0109940001630110000010021P8,671871,NaN
164811,None,IDCA,DAFW000024977,A0290660810673,ส่งมอบสินค้า,"{""Status"": ""00"", ""TotalTax"": ""33764.00"", ""Tota...",2023-09-01 04:17:47+00:00,ebxml_DAFW_RES_RESPONSE_A_DAFW000024977_202309...,None,TH0109940001630110000010021P8,NaN,NaN


In [13]:
df_response_import.to_csv('df_response_import.csv', index=False)


In [14]:
df_response_import.to_sql('response_import', postgres_mybase, if_exists='append', index=False)


813

In [ ]:
# from sqlalchemy import text

# # Update ic_is_use and ic_ms_messageid in database
# def update_database(cancel_df, table_name, columns_1, columns_2, pk_col, engine):
#     def update_target(conn, cancel_row):
#         update_query = text(f"""
#             UPDATE {table_name}
#             SET {columns_1} = :columns_1,
#                 {columns_2} = :columns_2
#             WHERE {pk_col} = :pk_col
#         """)
#         conn.execute(update_query, {
#             'columns_1': cancel_row[columns_1],
#             'columns_2': cancel_row[columns_2],
#             'pk_col': cancel_row[pk_col]
#         })

#     # Remove NaN values from columns_1 and columns_2
#     cancel_df = cancel_df.dropna(subset=[columns_1, columns_2])

#     with engine.connect() as conn:
#         trans = conn.begin()
#         try:
#             for _, cancel_row in cancel_df.iterrows():
#                 update_target(conn, cancel_row)
#             trans.commit()
#             print(f"Data successfully updated in {table_name}.")
#         except Exception as e:
#             trans.rollback()
#             print(f"An error occurred while updating {table_name}: {e}")

# update_database(df_response_import, 'response_import', 'res_inbox_id', 'res_message_id', 'res_filename', postgres_mybase)

